In [ ]:
import re
import matplotlib.pyplot as plt

In [ ]:
def read_log(log):
    with open(log, "r") as f:
        k = f.read()
    its = [[]]
    for n in re.findall("FGMRES: d\(\d*\)= (.*)",k):
        its[-1].append(n)
        if "rate" in n:
            its.append([])
    its = its[0],its[1]
    its = [[float(h) for h in g if "rate" not in h] for g in its]
    return its

In [ ]:
its0 = read_log("../spacetimeproject/build/log/log.21-02-03-12:18:47")
plt.semilogy(range(len(its0[0])),its0[0])
plt.semilogy(range(len(its0[1])),its0[1])

In [ ]:
its = read_log("../spacetimeproject/build/log/log")
plt.semilogy(range(len(its[0])),its[0])
plt.semilogy(range(len(its0[1])),its0[1])
plt.semilogy(range(len(its[1])),its[1])

In [ ]:
its = read_log("../spacetimeproject/build/log/log")
plt.semilogy(range(len(its[0])),its[0])
plt.semilogy(range(len(its[1])),its[1])
plt.semilogy(range(len(its0[1])),its0[1])

In [ ]:
its2 = read_log("../spacetimeproject/build/log/log")
plt.semilogy(range(len(its2[0])),its2[0])
plt.semilogy(range(len(its2[1])),its2[1])
plt.semilogy(range(len(its0[1])),its0[1])


In [ ]:
import sys
import numpy as np
import vtk
from vtk.util import numpy_support
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import os
from matplotlib import animation, rc
import re
import matplotlib.pyplot as plt
directory = "../spacetimeproject/build/data/vtk"

In [ ]:
rc('animation', html='html5')
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

In [ ]:
def load_f(idd, comp, ds):

    fn = directory + "/" + ds[comp][idd]

    reader = vtk.vtkUnstructuredGridReader()
    reader.SetFileName(fn)
    reader.Update()

    polydata = reader.GetOutput()
    data = polydata.GetCellData().GetArray('scalar_value')
    as_numpy = numpy_support.vtk_to_numpy(data)

    histo = {}
    for i in range(polydata.GetNumberOfCells()):
        pts = polydata.GetCell(i).GetPoints()    
        np_pts = np.array([pts.GetPoint(i) for i in range(pts.GetNumberOfPoints())])
        #data_ls.append(numpy.array([data.GetValue(i) for i in range(pts.GetNumberOfPoints())]))
        #print(np_pts)
        t = np.average(np_pts[:,2])
        if t not in histo:
            histo[t] = 0
        histo[t] += as_numpy[i]
    
    #print(histo)
    
    x = [k for k in sorted(histo)]
    y = [histo[z] for z in x]
    return x,y
#x,y = load_f(1,1)
#import matplotlib.pyplot as plt
#plt.plot(x,np.abs(y))

In [ ]:
def load_residual_history():

    ds = []
    for gg in ["P0_r1_", "V_x_r1_", "V_y_r1"]:
        k = [g for g in os.listdir(directory) if gg in g]
        k.sort(key=natural_keys)
        #print(k)
        ds.append(k)

    fig, ax = plt.subplots()

    ln = []
    for c, name in zip(["blue", "green", "red"],["P", "Vx", "Vy"]):
        l, = plt.plot([], [], color=c, label=name)
        ln.append(l)
    plt.legend()      

    def update(frame):   
        mm = 1e-10
        for comp in [0,1,2]:
            x,y = load_f(frame, comp, ds)
            ln[comp].set_data(x, np.abs(y))      
            #print(np.max(np.abs(y)))
            mm = max(mm,np.max(np.abs(y)))
        ax.set_xlim(x[0], x[-1])    
        ax.set_ylim(0, mm)
        return ln

    #ani = FuncAnimation(fig, update, frames=len(ds[0]), blit=True, interval=1000)
    #HTML(ani.to_html5_video())
    return FuncAnimation(fig, update, frames=len(ds[0]), blit=True, interval=1000)

In [ ]:
%matplotlib inline

In [ ]:
print("")
load_residual_history()